In [1]:
import pandas as pd
import numpy as np

In [2]:
shifts=["None","Float","0","-1","Security","Greet","Tickets","Lunch","Coro","Trike","Gallery"]
people=["Brian","Ross","Will","Alexandra","Max","Chella","Chloe","Rohan","Nhi","Nolan","Sook"]
shift_colors={
    "":"lightGray",
    "0":"red",
    "-1":"yellow",
    "Security":"blue",
    "Greet":"magenta",
    "Lunch":"gray",
    "Gallery":"green",
    "Coro":"#F5F5DC",
    "Trike":"cyan"
}

In [3]:
def minutes_to_hhmm(mins):
    hours = mins//60
    minutes = mins%60
    return f'{hours:02d}:{minutes:02d}'

def minutes_to_12h(mins):
    hours = mins//60
    minutes = mins%60
    if hours == 0:
        h = "12"
        ampm = "AM"
    elif hours < 12:
        h = f'{hours:02d}'
        ampm = "AM"
    elif hours == 12:
        h = "12"
        ampm = "PM"
    else:
        h2 = hours-12
        h = f'{h2:02d}'
        ampm = "PM"
    
    return h + f':{minutes:02d}' + ' ' + ampm

print(minutes_to_12h(720))

12:00 PM


In [4]:
# At each time and person, store a shift
start_time = 600
end_time = 1020
default_schedule = pd.DataFrame("Float", index=range(start_time,end_time,30),columns=people)

In [5]:
print(default_schedule.index[2])
print([i for i in range(2)])

660
[0, 1]


In [6]:
default_schedule['Brian'][600]="Gallery"

print(default_schedule)

       Brian   Ross   Will Alexandra    Max Chella  Chloe  Rohan    Nhi  \
600  Gallery  Float  Float     Float  Float  Float  Float  Float  Float   
630    Float  Float  Float     Float  Float  Float  Float  Float  Float   
660    Float  Float  Float     Float  Float  Float  Float  Float  Float   
690    Float  Float  Float     Float  Float  Float  Float  Float  Float   
720    Float  Float  Float     Float  Float  Float  Float  Float  Float   
750    Float  Float  Float     Float  Float  Float  Float  Float  Float   
780    Float  Float  Float     Float  Float  Float  Float  Float  Float   
810    Float  Float  Float     Float  Float  Float  Float  Float  Float   
840    Float  Float  Float     Float  Float  Float  Float  Float  Float   
870    Float  Float  Float     Float  Float  Float  Float  Float  Float   
900    Float  Float  Float     Float  Float  Float  Float  Float  Float   
930    Float  Float  Float     Float  Float  Float  Float  Float  Float   
960    Float  Float  Floa

In [7]:
from PyQt5.QtWidgets import QApplication
app = QApplication([])

In [8]:
from PyQt5.QtWidgets import QWidget, QMainWindow, QComboBox, QLabel, QPushButton, QLineEdit
from PyQt5.QtWidgets import QVBoxLayout, QHBoxLayout, QGridLayout
from PyQt5.QtWidgets import QToolBar, QAction, QStatusBar
from PyQt5.QtGui import QIcon, QPalette, QColor, QBrush
from PyQt5.QtCore import Qt, pyqtSignal

In [9]:
from functools import partial

class ShiftSelector(QWidget):
    shiftChanged = pyqtSignal(str)
    # textboxChanged = pyqtSignal(str)
    durationChanged = pyqtSignal(str)
    
    def __init__(self, person, start_time, shifts, durations=[], shift_colors=dict(), max_width = 5): 
        # TODO add scrolling features
        super(ShiftSelector, self).__init__()
        
        self.person = person
        self.start_time = start_time
        self.shifts = shifts
        self.durations = durations
        self.shift_colors = shift_colors
        if "" not in shift_colors:
            shift_colors[""] = "white"
        
        self.active = True
        
        # shift dropdown
        shift_box = QComboBox()
        shift_box.addItems(shifts)
        shift_box.addItem("Other")
        
        shift_box.currentTextChanged.connect(self.shift_changed)
        
        self.shift_box = shift_box
        
        # other shift textbox
        text_box = QLineEdit()
        text_box.setVisible(False)
        
        text_box.textChanged.connect(self.textbox_changed)
        
        self.text_box = text_box
        
        # duration dropdown
        duration_box = QComboBox()
        duration_box.addItems(durations)
        
        duration_box.currentTextChanged.connect(self.duration_changed)
        
        self.duration_box = duration_box
        
        # layout
        leftSide = QVBoxLayout()
        leftSide.addWidget(shift_box)
        leftSide.addWidget(text_box)
        leftSide.setContentsMargins(5,5,5,5)
        leftWidget = QWidget()
        leftWidget.setLayout(leftSide)
        
        rightSide = QVBoxLayout()
        rightSide.addWidget(QLabel("Duration"))
        rightSide.addWidget(duration_box)
        rightSide.setContentsMargins(0,5,5,5)
        rightWidget = QWidget()
        rightWidget.setLayout(rightSide)
        
        mainLayout = QHBoxLayout()
        mainLayout.addWidget(leftWidget)
        mainLayout.addWidget(rightWidget)
        mainLayout.setContentsMargins(0,0,0,0)
        
        self.mainLayout = mainLayout
        self.simpleLayout = leftSide
        
        self.setLayout(self.simpleLayout)
        
        self.setAutoFillBackground(True)
        palette = self.palette()
        palette.setColor(QPalette.Window, QColor("white")) # Magic Word
        self.setPalette(palette)
    
    # TODO: add color changing to shift_changed/textbox_changed 
    
    def set_shift(self,shift):
        if shift in self.shifts:
            self.shift_box.setCurrentText(shift)
        else:
            self.shift_box.setCurrentText("Other") # Magic Word
            self.text_box.setText(shift)
    
    def get_shift(self):
        if self.shift_box.currentText() == "Other":
            return self.text_box.currentText()
        else:
            return self.shift_box.currentText()
    
    def shift_changed(self, shift):
        if shift == "Other":
            self.text_box.setEnabled(True)
            self.text_box.setVisible(True)
        else:
            self.text_box.setEnabled(False)
            self.text_box.setVisible(False)
            self.shiftChanged.emit(shift)
        
        self.change_color_by_shift(shift)
            
    def textbox_changed(self, text):
        self.shiftChanged.emit(text)
        self.change_color_by_shift(text)
    
    def duration_changed(self, duration):
        self.durationChanged.emit(duration)
    
    def change_color_by_shift(self, shift):
        color = QColor(shift_colors[""])
        if shift in shift_colors:
            color = QColor(shift_colors[shift])
        palette = self.palette()
        palette.setColor(QPalette.Window, color)
        self.setPalette(palette)
        

In [34]:
class CentralGrid(QWidget):
    def __init__(self, schedule, shifts, shift_colors=dict(), max_width=-1, max_height=-1):
        super(CentralGrid, self).__init__()
        
        self.schedule = schedule
        self.shifts = shifts
        self.shift_colors = shift_colors
        
        people = list(schedule.columns)
        self.people = people
        self.people_index = 0
        times = list(schedule.index)
        self.times = times
        self.time_index = 0
        
        if max_width > 0:
            self.max_width = min(max_width,len(people))
        else:
            self.max_width = len(people)
            
        if max_height > 0:
            self.max_height = min(max_height,len(times))
        else:
            self.max_height = len(times)
        
        widget_grid = dict()
        name_widgets = dict()
        time_widgets = dict()
        
        # Names at the top
        for i in range(schedule.shape[1]):
            name = people[i]
            name_widget = QLabel(name)
            
            name_widget.setAutoFillBackground(True)
            palette = name_widget.palette()
            palette.setColor(QPalette.Window, QColor("white"))
            name_widget.setPalette(palette)
            
            name_widgets[name] = name_widget
            
            widget_grid[people[i]] = dict() # initialize the interior of the widget storage
        
        # Times at the left
        for i in range(schedule.shape[0]):
            time = schedule.index[i]
            formatted_time = minutes_to_12h(time)
            time_widget = QLabel(formatted_time)
            
            time_widget.setAutoFillBackground(True)
            palette = time_widget.palette()
            palette.setColor(QPalette.Window, QColor("white"))
            time_widget.setPalette(palette)
            
            time_widgets[time] = time_widget
        
        # Dropdowns in the middle
        # TODO encapsulate the layout creation
        for i in range(len(times)):
            time = times[i]
            
            durations = [repr(30+times[k]-time) for k in range(i,len(times))] # this is very scuffed lol
            for j in range(len(people)):
                person = people[j]
                
                widget = ShiftSelector(person, time, shifts, durations, shift_colors=shift_colors)
                widget.set_shift(self.schedule[person][time])
                
                widget.shiftChanged.connect(partial(self.updateSchedule,person,time))
                widget.shiftChanged.connect(lambda x : self.updateCounters())
                widget_grid[person][time] = widget
        
        main_height = self.max_height+1
        main_width = self.max_width+1
        
        # Column shift counters
        colCounters = {p:QLabel("") for p in schedule.columns} # TODO just make the list of people an input to the grid
        for l in colCounters.values():
            l.setAutoFillBackground(True)
            palette = l.palette()
            palette.setColor(QPalette.Window, QColor("white"))
            l.setPalette(palette)
        colShift = ShiftSelector(None, None, shifts, [])
        
        colShift.shiftChanged.connect(lambda x : self.updateCounters())
        self.colShift = colShift
        self.colCounters = colCounters
            
        # Row shift counters
        rowCounters = {t:QLabel("") for t in schedule.index} # TODO make the list of times an input as well?
        for l in rowCounters.values():
            l.setAutoFillBackground(True)
            palette = l.palette()
            palette.setColor(QPalette.Window, QColor("white"))
            l.setPalette(palette)
        rowShift = ShiftSelector(None, None, shifts, [])
        
        rowShift.shiftChanged.connect(lambda x : self.updateCounters())
        self.rowShift = rowShift
        self.rowCounters = rowCounters
        
        # TEMPORARY: add a "print" button in the top-left
        # Maybe change this to a save button later?
        print_button = QPushButton("Print")
        print_button.pressed.connect(self.printSchedule)
        self.print_button = print_button
        
        self.widget_grid = widget_grid
        self.name_widgets = name_widgets
        self.time_widgets = time_widgets
        
        self.setAutoFillBackground(True)
        palette = self.palette()
        palette.setColor(QPalette.Window, QColor("darkBrown"))
        self.setPalette(palette)
        
        self.updateLayout(people[:self.max_width],times[:self.max_height])
        self.updateCounters()
    
    def updateSchedule(self, person, time, shift):
        self.schedule[person][time] = shift
        # TODO update counters here as well
        # TODO add something about durations?
        
    def updateCounters(self):
        colShift = self.colShift.get_shift()
        for p in self.colCounters:
            val_counts = self.schedule[p].value_counts()
            if colShift in val_counts:
                self.colCounters[p].setText(repr(val_counts[colShift]))
            else:
                self.colCounters[p].setText("0")
        
        rowShift = self.rowShift.get_shift()
        for t in self.rowCounters:
            val_counts = self.schedule.loc[t].value_counts()
            if rowShift in val_counts:
                self.rowCounters[t].setText(repr(val_counts[rowShift]))
            else:
                self.rowCounters[t].setText("0")
    
    def updateLayout(self, people=None, times=None):
        if people == None:
            people = self.people[self.people_index:self.people_index+max_width]
        if times == None:
            times = self.times[self.time_index:self.time_index+max_width]
        
        newLayout = QGridLayout()
        main_width = len(people)
        main_height = len(times)
        
        newLayout.addWidget(self.print_button,0,0)
        newLayout.addWidget(self.rowShift,0,main_width+1)
        newLayout.addWidget(self.colShift,main_height+1,0)
        for i in range(len(times)):
            newLayout.addWidget(self.time_widgets[times[i]],i+1,0)
            newLayout.addWidget(self.rowCounters[times[i]],i+1,main_width+1)
        for i in range(len(people)):
            newLayout.addWidget(self.name_widgets[people[i]],0,i+1)
            newLayout.addWidget(self.colCounters[people[i]],main_height+1,i+1)
        
        for i in range(len(times)):
            for j in range(len(people)):
                newLayout.addWidget(self.widget_grid[people[j]][times[i]],i+1,j+1)
                
        if self.people_index+self.max_width < len(self.people):
            pass # TODO add an arrow
        
        newLayout.setSpacing(5)
        # self.layout_dp[(people,times)] = newLayout # does this even work???
        if self.layout():
            QWidget().setLayout(self.layout()) # apparently this basically deletes the old layout as long as we dont keep the reference to it
        self.setLayout(newLayout)
    
    def printSchedule(self):
        print(self.schedule)

In [ ]:
class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        
        self.setWindowTitle("MoMath Schedule Manager")
        
        widget = CentralGrid(default_schedule, shifts, shift_colors=shift_colors)
        
        self.setCentralWidget(widget)
        # TODO change default margins
w = MainWindow()
w.show()

app.exec()

In [ ]:
# Condition format: (1) every P [from list] has at least N S; (2) every T [in interval/list] has at least N S

In [18]:
None == None

True